## 유클리드 거리

In [3]:
import numpy as np
user1 = np.array([0, 1, 2, 3, 4, 5])
user2 = np.array([0, 1, 4, 6, 1, 4])

# 유클리드 거리 : 거리가 짧을수록 유사하다.
sum((user1 - user2) ** 2) ** (1/2) #두 벡터 사이의 거리

4.795831523312719

## 코사인 유사도

In [6]:
import numpy as np

user1 = np.array([4, 5, 3, 2, 1])
user2 = np.array([3, 2, 4, 2, 2])

In [14]:
a = sum(user1 * user2) #분자
print(a)

b = (sum(user1 ** 2) ** (1/2)) * (sum(user2 ** 2) ** (1/2)) #분모
print(b)

result = a / b
print(result)

51
62.048368229954285
0.8219394233058879


## PPT 데이터 이용

In [21]:
import numpy as np

A = np.array([
    [3, 3, 2, 3, 1],
    [5, 2, 2, 3, 1],
    [3, 3, 2, 3, 1],
    [3, 1, 4, 3, 1],
])
np.sum(A) #누락값이 있으면 계산 불가능
np.nansum(A) #누락값이 있어도 계산 가능

49

In [24]:
A = np.array([[4,1,5,np.nan,1],
     [2,3,np.nan,2,3],
     [1,np.nan,4,1,3],
     [np.nan,2,4,np.nan,2],
     [1,np.nan,4,1,3]])

#누랄 값 0으로 채우기
np.nan_to_num(A, nan = 0)

array([[4., 1., 5., 0., 1.],
       [2., 3., 0., 2., 3.],
       [1., 0., 4., 1., 3.],
       [0., 2., 4., 0., 2.],
       [1., 0., 4., 1., 3.]])

In [29]:
# 유저별 평균으로 채우기
A_copy =  A.copy()

for i in range(len(A_copy)):
    mean = np.nanmean(A_copy[i])
    A_copy[i] = np.nan_to_num(A_copy[i], nan = mean)
    A_copy[i] = A_copy[i] - mean # Mean Normalization으로 누락값 채우기


A_copy

array([[ 1.25      , -1.75      ,  2.25      ,  0.        , -1.75      ],
       [-0.5       ,  0.5       ,  0.        , -0.5       ,  0.5       ],
       [-1.25      ,  0.        ,  1.75      , -1.25      ,  0.75      ],
       [ 0.        , -0.66666667,  1.33333333,  0.        , -0.66666667],
       [-1.25      ,  0.        ,  1.75      , -1.25      ,  0.75      ]])

## 데이터 사용

In [1]:
import pandas as pd

ratings = pd.read_csv('data/ratings.csv')
df = ratings.pivot_table(index = 'userId', columns = 'movieId', values = 'rating', aggfunc = 'sum').values
df

array([[4. , nan, 4. , ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [2.5, 2. , 2. , ..., nan, nan, nan],
       [3. , nan, nan, ..., nan, nan, nan],
       [5. , nan, nan, ..., nan, nan, nan]])

In [9]:
nan_mask = np.isnan(df) #누락값 있는 곳 미리 저장

df_filled = df.copy()

for i in range(len(df_filled)):
    mean = np.nanmean(df_filled[i]) #각각의 유저들의 평균값
    df_filled[i] = np.nan_to_num(df_filled[i], nan = mean)
    df_filled[i] -= mean
df_filled

array([[-0.36637931,  0.        , -0.36637931, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.63417569, -1.13417569, -1.13417569, ...,  0.        ,
         0.        ,  0.        ],
       [-0.27027027,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.31144393,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [12]:
from sklearn.metrics.pairwise import euclidean_distances

#유클리드 거리 계산
distance_matrix = euclidean_distances(df_filled)
distance_matrix

array([[ 0.        , 12.88018162, 17.71367592, ..., 32.52330461,
        12.52374725, 33.10732342],
       [12.88018162,  0.        , 13.57431978, ..., 31.40943515,
         5.18205377, 31.11334478],
       [17.71367592, 13.57431978,  0.        , ..., 33.81269431,
        13.1676513 , 33.27108605],
       ...,
       [32.52330461, 31.40943515, 33.81269431, ...,  0.        ,
        31.07357893, 42.64413263],
       [12.52374725,  5.18205377, 13.1676513 , ..., 31.07357893,
         0.        , 31.07797619],
       [33.10732342, 31.11334478, 33.27108605, ..., 42.64413263,
        31.07797619,  0.        ]])

In [15]:
# 고려할 이웃의 수
k = 5 #다섯명정도만 골라봄.

#각 유저별로 가장 거리가 짧았던 5명이 있는 위치
nearest_neighbors = np.argsort(distance_matrix, axis = 1)[:, 1:k+1] #자기 자신을 빼고 5명 고려
nearest_neighbors

array([[ 52,  48, 188, 213, 514],
       [ 52, 188,  48, 514,  24],
       [ 52,  48, 514, 188,  24],
       ...,
       [315, 430, 539,  71,  53],
       [ 52,  48,  53, 514, 188],
       [462, 361, 347, 292, 121]], dtype=int64)

In [39]:
movies = pd.read_csv('data/movies.csv')
movie_names = {} #영화이름 저장

for i in range(len(movies)):
    a = movies.iloc[i]
    movie_names[a['movieId']] = a['title']

In [52]:
# 0번째 사람의 영화를 추천할 예정
user = 0
user_ratings = df[user] #해당 유저의 데이터
user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
neighbor_indices = nearest_neighbors[user] #유저들과 가장 가까운 데이터
unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
    neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
    for neighbor_idx in neighbor_indices:
        if not (nan_mask[neighbor_idx][movie_idx]):
            neighbor_ratings.append(df[neighbor_idx][movie_idx])
    if neighbor_ratings != []: #비어있지 않다면
        predictions[movie_idx] = np.mean(neighbor_ratings)

#키와 벨류값 묶어서 정렬
sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
for i in sorted_predictions[:3]:
    print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')

추천영화 : Jeffrey (1995) (예측평점 : 5.0점)
추천영화 : Burnt by the Sun (Utomlyonnye solntsem) (1994) (예측평점 : 5.0점)
추천영화 : Virtuosity (1995) (예측평점 : 5.0점)


In [55]:
def movie_reco(user):
    user_ratings = df[user] #해당 유저의 데이터
    user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
    neighbor_indices = nearest_neighbors[user] #유저들과 가장 가까운 데이터
    unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

    predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

    for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
        neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
        for neighbor_idx in neighbor_indices:
            if not (nan_mask[neighbor_idx][movie_idx]):
                neighbor_ratings.append(df[neighbor_idx][movie_idx])
        if neighbor_ratings != []: #비어있지 않다면
            predictions[movie_idx] = np.mean(neighbor_ratings)

    #키와 벨류값 묶어서 정렬
    sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
    for i in sorted_predictions[:3]:
        print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')
movie_reco(0)

추천영화 : Jeffrey (1995) (예측평점 : 5.0점)
추천영화 : Burnt by the Sun (Utomlyonnye solntsem) (1994) (예측평점 : 5.0점)
추천영화 : Virtuosity (1995) (예측평점 : 5.0점)


In [59]:
box = [[1,2], [3,1], [5,7], [10, 0]]

def func1(x):
    return x[0]

In [60]:
box.sort(key = func1)
box

[[1, 2], [3, 1], [5, 7], [10, 0]]

In [61]:
box.sort(key = lambda x : x[1])
box

[[10, 0], [3, 1], [1, 2], [5, 7]]

In [62]:
func1 = lambda x : x[1]

In [63]:
func1([1,2])

2